In [49]:
from pyspark.sql import SparkSession
import csv
import pandas as pd

spark = SparkSession.builder.master("local").appName("lopputyo").getOrCreate()

df = spark.read.csv("merged10_21.csv", header=True, inferSchema=True)
df.show(vertical=True)

-RECORD 0-----------------------------
 id                 | 0               
 aikatyyppi         | Vuosi           
 kuukausi_nro       | 12              
 vuosikuukausi      | 201212          
 vuosi              | 2012            
 kunta_nro          | 239             
 kunta_nimi         | Keitele         
 ikaryhma           | 30-34           
 sukupuoli          | Mies            
 etuus              | Työmarkkinatuki 
 korvausperuste     | Yhteensä        
 saaja_lkm          | 5               
 korvattu_paiva_lkm | 614             
 maksettu_eur       | 21222.96        
-RECORD 1-----------------------------
 id                 | 1               
 aikatyyppi         | Vuosi           
 kuukausi_nro       | 12              
 vuosikuukausi      | 201212          
 vuosi              | 2012            
 kunta_nro          | 236             
 kunta_nimi         | Kaustinen       
 ikaryhma           | 30-34           
 sukupuoli          | Mies            
 etuus              | Työ

In [55]:
df.printSchema()

root
 |-- id: integer (nullable = true)
 |-- aikatyyppi: string (nullable = true)
 |-- kuukausi_nro: integer (nullable = true)
 |-- vuosikuukausi: integer (nullable = true)
 |-- vuosi: integer (nullable = true)
 |-- kunta_nro: integer (nullable = true)
 |-- kunta_nimi: string (nullable = true)
 |-- ikaryhma: string (nullable = true)
 |-- sukupuoli: string (nullable = true)
 |-- etuus: string (nullable = true)
 |-- korvausperuste: string (nullable = true)
 |-- saaja_lkm: integer (nullable = true)
 |-- korvattu_paiva_lkm: integer (nullable = true)
 |-- maksettu_eur: double (nullable = true)



In [33]:
df.registerTempTable("example")
df.groupBy('sukupuoli').count().orderBy('count',ascending=False).show()

+----------+-------+
| sukupuoli|  count|
+----------+-------+
|      Mies|3244082|
|    Nainen|3207493|
|Tuntematon| 737123|
+----------+-------+



In [34]:
from pyspark.sql import functions as F

df.select(F.col("etuus"))

DataFrame[etuus: string]

In [35]:
df.select("sukupuoli").distinct().count()

3

In [61]:
#new_df = df.withColumn("maksu_per_hlo", (df["maksettu_eur"]/df["saaja_lkm"])).orderBy('id',ascending=False).filter(df["aikatyyppi"] == "Kuukausi")
new_df = df.withColumn("maksu_per_hlo", (df["maksettu_eur"]/df["saaja_lkm"])).orderBy('id',ascending=False).filter(df["aikatyyppi"] == "Vuosi")
new_df.show()

SyntaxError: invalid syntax (3175812245.py, line 3)

In [60]:
new_df.select("vuosi", "maksu_per_hlo").groupBy("vuosi").avg().orderBy("vuosi").show()

+-----+----------+------------------+
|vuosi|avg(vuosi)|avg(maksu_per_hlo)|
+-----+----------+------------------+
| 2010|    2010.0| 2962.980747694528|
| 2011|    2011.0| 3027.965801504773|
| 2012|    2012.0|3675.9797612511948|
| 2013|    2013.0|3828.1058665663195|
| 2014|    2014.0| 3986.256119863563|
| 2015|    2015.0| 3982.663336442664|
| 2016|    2016.0|4133.9824763662045|
| 2017|    2017.0|3989.0794887144584|
| 2018|    2018.0|1950.6392863797682|
| 2019|    2019.0|1954.1727660418715|
| 2020|    2020.0|1972.7234112020496|
| 2021|    2021.0|2094.3429509914777|
+-----+----------+------------------+



In [54]:
from pyspark.sql import functions as F
yes = new_df.filter((F.col('vuosi')=='2017')).show()


#new_df.groupBy("kunta_nimi").avg("maksu_per_hlo").orderBy("avg(maksu_per_hlo)", ascending=False).show()

+---+----------+------------+-------------+-----+---------+----------+--------+---------+-----+--------------+---------+------------------+------------+-------------+
| id|aikatyyppi|kuukausi_nro|vuosikuukausi|vuosi|kunta_nro|kunta_nimi|ikaryhma|sukupuoli|etuus|korvausperuste|saaja_lkm|korvattu_paiva_lkm|maksettu_eur|maksu_per_hlo|
+---+----------+------------+-------------+-----+---------+----------+--------+---------+-----+--------------+---------+------------------+------------+-------------+
+---+----------+------------+-------------+-----+---------+----------+--------+---------+-----+--------------+---------+------------------+------------+-------------+

